<a href="https://colab.research.google.com/github/LucasMartinscode/Python/blob/LucasMartinscode-patch-1/Atividade%20aula%2007.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Atividade 06 - Engenharia de dados

In [3]:
#Instalando as bibliotecas necessárias

!pip install pymongo
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 7.9 MB/s eta 0:00:00


In [36]:
#Importando as libs
import numpy as np
import pandas as pd
from io import BytesIO
from pymongo import MongoClient
import boto3
from io import StringIO
import json
import os

In [58]:
# Configurar o cliente MinIO
client = boto3.client('s3',
    endpoint_url='http://awari-minio-nginx:9000',
    aws_access_key_id='pFUkmwZQujAl38nr',
    aws_secret_access_key='OE62cNwuoGQwytwJqL6OhljvnfxHSwdP',
    aws_session_token=None,
    config=Config(signature_version='s3v4'),
    verify=False,
    region_name='sa-east-1'
)

In [38]:
#Manipulando os dados
#Entendendo os estados
data_es = pd.read_csv('estados.csv')
print (data_es.head())

   codigo_uf  uf      nome  latitude  longitude regiao
0         11  RO  Rondônia    -10.83     -63.34  Norte
1         12  AC      Acre     -8.77     -70.55  Norte
2         13  AM  Amazonas     -3.47     -65.10  Norte
3         14  RR   Roraima      1.99     -61.33  Norte
4         15  PA      Pará     -3.79     -52.48  Norte


In [39]:
#Entendendo municipios

data_mun = pd.read_csv('municipios.csv')
print (data_mun.head())

   codigo_ibge                 nome  latitude  longitude  capital  codigo_uf  \
0      5200050      Abadia de Goiás -16.75730   -49.4412        0         52   
1      3100104  Abadia dos Dourados -18.48310   -47.3916        0         31   
2      5200100            Abadiânia -16.19700   -48.7057        0         52   
3      3100203               Abaeté -19.15510   -45.4444        0         31   
4      1500107           Abaetetuba  -1.72183   -48.8788        0         15   

   siafi_id  ddd       fuso_horario  
0      1050   62  America/Sao_Paulo  
1      4001   34  America/Sao_Paulo  
2      9201   62  America/Sao_Paulo  
3      4003   37  America/Sao_Paulo  
4       401   91  America/Sao_Paulo  


In [40]:
#Realizando a junção

Ambas_tabelas = pd.merge(data_mun, data_es, on='codigo_uf', how='inner')
print (Ambas_tabelas.head())


   codigo_ibge              nome_x  latitude_x  longitude_x  capital  \
0      5200050     Abadia de Goiás    -16.7573     -49.4412        0   
1      5200100           Abadiânia    -16.1970     -48.7057        0   
2      5200134             Acreúna    -17.3960     -50.3749        0   
3      5200159           Adelândia    -16.4127     -50.1657        0   
4      5200175  Água Fria de Goiás    -14.9778     -47.7823        0   

   codigo_uf  siafi_id  ddd       fuso_horario  uf nome_y  latitude_y  \
0         52      1050   62  America/Sao_Paulo  GO  Goiás      -15.98   
1         52      9201   62  America/Sao_Paulo  GO  Goiás      -15.98   
2         52      9645   64  America/Sao_Paulo  GO  Goiás      -15.98   
3         52      9769   64  America/Sao_Paulo  GO  Goiás      -15.98   
4         52      9771   62  America/Sao_Paulo  GO  Goiás      -15.98   

   longitude_y        regiao  
0       -49.86  Centro-Oeste  
1       -49.86  Centro-Oeste  
2       -49.86  Centro-Oeste  
3   

In [47]:
#Salvando data set's por UF

for index, row in Ambas_tabelas.iterrows():
    sigla = row['uf']  # Ajuste 'uf' se o nome da coluna for diferente
    if not os.path.exists(sigla):
        os.makedirs(sigla)

In [49]:
# prompt:

colunas_selecionadas = ['uf', 'nome_x']  # ajuste os nomes das colunas conforme necessário
dados_filtrados = Ambas_tabelas[colunas_selecionadas]

# Renomear colunas para um padrão mais claro
dados_filtrados.columns = ['UF', 'Cidade']

# Ordenar por UF e Cidade
dados_filtrados = dados_filtrados.sort_values(by=['UF', 'Cidade'])

# Salvar o resultado em um novo arquivo CSV
dados_filtrados.to_csv('cidades.csv', index=False)

print("Arquivo cidades.csv criado com sucesso!")

Arquivo cidades.csv criado com sucesso!


In [50]:
# salvar esse arquivo cidades.csv dentro da pasta da UF (estado) correspondente.

for uf, grupo in dados_filtrados.groupby('UF'):
    if not os.path.exists(uf):
        os.makedirs(uf)
    caminho_arquivo = os.path.join(uf, 'cidades.csv')
    grupo.to_csv(caminho_arquivo, index=False)

print("Arquivos cidades.csv criados dentro das respectivas pastas de UF com sucesso!")

Arquivos cidades.csv criados dentro das respectivas pastas de UF com sucesso!


In [62]:
bucket_name = 'aula-06'
prefix = 'Estados/cidade/'

# Função para fazer upload dos arquivos
def upload_files(folder, client, bucket_name, prefix):
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith('.csv'):
                file_path = os.path.join(root, file)

                # Ler o arquivo CSV
                df = pd.read_csv(file_path)

                # Salvar no buffer
                csv_buffer = StringIO()
                df.to_csv(csv_buffer, index=False)

                # Criar chave para o objeto no bucket
                key = os.path.join(prefix, os.path.relpath(file_path, folder)).replace("\\", "/")

                # Upload para o bucket
                client.put_object(Body=csv_buffer.getvalue(), Bucket=bucket_name, Key=key)
                print(f'Uploaded {file_path} to {key}')

# Pasta onde as pastas UF estão localizadas
base_folder = '.'  # ou o caminho relativo onde as pastas UF estão

# Fazer upload dos arquivos
upload_files(base_folder, client, bucket_name, prefix)

print("Arquivos importados para o bucket no MinIO com sucesso!")

"Não estou conseguindo acessar o endpoint url da awari"

EndpointConnectionError: Could not connect to the endpoint URL: "http://awari-minio-nginx:9000/aula-06/Estados/cidade/municipios.csv"